In [ ]:
import cv2
import os
from PIL import Image
from scipy import misc
import tensorflow as tf
import tensorflow.keras as keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import gzip
%matplotlib inline
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img


# DATA Preprocessing

In [ ]:
#Data Preprocessing

def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
    
    for file in os.listdir(img_folder):
        
        image_path = os.path.join(img_folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)
        img_data_array.append(image)
        class_name.append(int(file[0]))
        
    return np.array(img_data_array), np.array(class_name)

def normalization(image):
    image = image / image.max()
    return image


training_path = "./01_finger_training"
test_path = './01_finger_test'

train_data, train_label = create_dataset(training_path)
test_data, _ = create_dataset(test_path)

train_data = normalization(train_data)
test_data = normalization(test_data)

In [ ]:
#preparing test_result_DataFrame

test_index = os.listdir(test_path)
temp = []

for index in test_index:
    index = index.split(".")
    temp.append(int(index[0]))
    
test_index = temp

In [ ]:
print('Shape of Train images :',train_data.shape)
print('Shape of Train labels : ', train_label.shape)
print('Shape of Test images : ', test_data.shape)

# Model

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(144, 144)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(9, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_label, epochs=10)

# Training Validation

In [ ]:
predictions = model.predict(train_data)

In [ ]:
print(predictions.shape)

In [ ]:
predict_label = []
for image in predictions:
    predict_label.append(np.argmax(image))

In [ ]:
predict_label = np.array(predict_label)

In [ ]:
def accuracy(original, prediction):
    accuracy = original == prediction
    accuracy = np.count_nonzero(accuracy)
    
    return accuracy / original.shape[0]
    
accuracy(train_label, predict_label)

# Test

In [ ]:
predictions = model.predict(test_data)

In [ ]:
print(predictions.shape)

In [ ]:
test_label = []
for image in predictions:
    test_label.append(np.argmax(image))

In [ ]:
test_label = np.array(test_label)

In [ ]:
#결과
print(test_label)

# Extract Pandas DataFrame

In [ ]:
test_result_df = pd.DataFrame([x for x in zip(test_index, test_label)], columns=['Image', 'Answer'])

In [ ]:
test_result_df.head()

In [ ]:
test_result_df = test_result_df.sort_values(by="Image")
test_result_df.head()

In [ ]:
test_result_df.to_csv("./result.csv", index=False)